### Import necessary libraries:

In [4]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

### Set up data generators and load the dataset:

In [ ]:
# Dataset preparation
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'data/val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)

### Define base models:

In [ ]:
base_models = {
    'VGG16': VGG16(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,)),
    'ResNet50': ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,)),
    'InceptionV3': InceptionV3(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
}

### Define the fine-tuning function:

In [ ]:
def fine_tune_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### Define the training and evaluation function:

In [ ]:
def train_and_evaluate(model, model_name):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        validation_data=val_generator,
        validation_steps=val_generator.samples // BATCH_SIZE,
        epochs=10
    )

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test_generator)

    # Calculate precision, recall, and F1 score
    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes

    precision = precision_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')
    f1 = f1_score(y_true, y_pred_classes, average='weighted')

    return {
        'model': model_name,
        'accuracy': test_accuracy,
        'loss': test_loss,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'history': history
    }

### Train and evaluate models:

In [ ]:
results = {}
for name, base_model in base_models.items():
    print(f"Training and evaluating {name}...")
    model = fine_tune_model(base_model, num_classes)
    results[name] = train_and_evaluate(model, name)

### Display results table:

In [ ]:
print("\nModel Performance Comparison:")
print("Model\t\tAccuracy\tLoss\t\tPrecision\tRecall\t\tF1 Score")
print("-" * 80)
for name, result in results.items():
    print(f"{name}\t\t{result['accuracy']:.4f}\t\t{result['loss']:.4f}\t\t{result['precision']:.4f}\t\t{result['recall']:.4f}\t\t{result['f1_score']:.4f}")

### Plot training history:

In [ ]:
plt.figure(figsize=(15, 5))
for name, result in results.items():
    plt.plot(result['history'].history['accuracy'], label=f'{name} Training Accuracy')
    plt.plot(result['history'].history['val_accuracy'], label=f'{name} Validation Accuracy')

plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### Print fine-tuning process description:

In [ ]:
print("\nFine-tuning Process:")
print("1. I started with pre-trained models (VGG16, ResNet50, InceptionV3) and froze their base layers.")
print("2. I added a Global Average Pooling layer to reduce the spatial dimensions.")
print("3. I added a dense layer with 1024 units and ReLU activation for feature extraction.")
print("4. I added a final dense layer with softmax activation for classification.")
print("5. I used a small learning rate (0.0001) to fine-tune the model without disrupting the pre-trained weights.")
print("6. I used data augmentation techniques to increase the diversity of our training data.")
print("7. I trained the models for 10 epochs, which can be adjusted based on performance.")